In [2]:
# 데이터 불러오기
df = read.csv('../data/petfinder-pawpularity-score/train.csv')
head(df)

Id,Subject.Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72
001dd4f6fafb890610b1635f967ea081,0,0,1,0,0,0,0,0,0,0,0,1,74


In [4]:
# 회귀분석
model <- lm(Pawpularity~ Subject.Focus+Eyes+Face+Near+Action+Accessory+Group+Collage+Human+Occlusion+Info+Blur,data = df) # ~. 는 나머지 모든 변수에 대해 회귀분석 실시
summary(model)


Call:
lm(formula = Pawpularity ~ Subject.Focus + Eyes + Face + Near + 
    Action + Accessory + Group + Collage + Human + Occlusion + 
    Info + Blur, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-39.509 -12.839  -4.809   8.191  66.630 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)   37.37044    0.87220  42.846  < 2e-16 ***
Subject.Focus -1.12585    1.27301  -0.884 0.376503    
Eyes          -2.48010    0.74920  -3.310 0.000935 ***
Face           2.45014    0.91795   2.669 0.007617 ** 
Near           0.46845    0.65948   0.710 0.477519    
Action        -0.30890    2.08153  -0.148 0.882030    
Accessory      1.21101    0.82972   1.460 0.144448    
Group          1.22000    0.65853   1.853 0.063968 .  
Collage        0.39700    1.12308   0.353 0.723724    
Human          0.36363    0.72889   0.499 0.617879    
Occlusion     -0.06627    0.71858  -0.092 0.926519    
Info          -0.71554    0.99384  -0.720 0.471562    
Blur        

- Pr(>|t|)열은 t 분포를 사용하여 각 변수가 얼마나 유의한지를 판단할 수 있는 p-value를 알려줌
- Pr(>|t|) 값이 0.05보다 큰 값은 회귀분석모형에서 빠지는게 유력한 변수

In [5]:
# stepwise 방법으로 변수 선택하기
model2 <- step(model, direction = "both") # 단계별 선택법

Start:  AIC=59966.75
Pawpularity ~ Subject.Focus + Eyes + Face + Near + Action + Accessory + 
    Group + Collage + Human + Occlusion + Info + Blur

                Df Sum of Sq     RSS   AIC
- Occlusion      1       3.6 4192441 59965
- Action         1       9.3 4192446 59965
- Collage        1      52.9 4192490 59965
- Human          1     105.4 4192542 59965
- Near           1     213.7 4192651 59965
- Info           1     219.5 4192656 59965
- Subject.Focus  1     331.3 4192768 59966
<none>                       4192437 59967
- Accessory      1     902.2 4193339 59967
- Group          1    1453.6 4193891 59968
- Face           1    3017.3 4195454 59972
- Eyes           1    4641.1 4197078 59976
- Blur           1    6053.4 4198490 59979

Step:  AIC=59964.76
Pawpularity ~ Subject.Focus + Eyes + Face + Near + Action + Accessory + 
    Group + Collage + Human + Info + Blur

                Df Sum of Sq     RSS   AIC
- Action         1       9.3 4192450 59963
- Collage        1      53

- Eyes , Face , Group , Blur를 독립변수로 선정?

In [6]:
# 회귀모델 비교 및 평가
anova(model, model2)

Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
9899,4192437,NA,NA,NA,NA
9907,4194227,-8,-1790.027,0.5283167,0.8360922


In [7]:
summary(model2)


Call:
lm(formula = Pawpularity ~ Eyes + Face + Group + Blur, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-39.308 -12.856  -4.856   8.144  65.935 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  37.8043     0.6967  54.264  < 2e-16 ***
Eyes         -2.4421     0.7447  -3.279 0.001044 ** 
Face          2.4937     0.9163   2.721 0.006512 ** 
Group         1.0104     0.6198   1.630 0.103114    
Blur         -3.7389     0.9919  -3.769 0.000165 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 20.58 on 9907 degrees of freedom
Multiple R-squared:  0.001983,	Adjusted R-squared:  0.00158 
F-statistic: 4.922 on 4 and 9907 DF,  p-value: 0.0005801


In [12]:
# 다중공선성 확인

# 다중공선성은 분산팽창지수(Variation Inflation Factor;VIF)라는 통계량을 사용하여 계산 가능
# 한 예측변수에 대해 VIF의 제곱근은 다중공선선의 정도를 나타냄
# VIF가 10을 넘으면(=GVIF^(1/(2*Df)) 값이 2를 넘으면), 다중공선성 문제를 보임

# VIF 값은 car 패키지의 vif() 함수로 계산
install.packages("car")
library(car)
# vif(model)

also installing the dependency 'lme4'




  There are binary versions available but the source versions are later:
     binary   source needs_compilation
lme4 1.1-26 1.1-27.1              TRUE
car  3.0-10   3.0-12             FALSE

  Binaries will be installed
package 'lme4' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\tmp\Rtmpsv98Vh\downloaded_packages


installing the source package 'car'



In [13]:
# 팽창계수 확인
vif(model)

Subject.Focus          Eyes          Face          Near        Action 
     1.019446      2.307956      1.712161      1.213886      1.002690 
    Accessory         Group       Collage         Human     Occlusion 
     1.018282      1.142908      1.392512      1.723615      1.721186 
         Info          Blur 
     1.328935      1.510249

In [14]:
vif(model2)

Eyes     Face    Group     Blur 
2.281203 1.706776 1.012895 1.507808

In [15]:
model3 <- lm(Pawpularity~ Subject.Focus+Face+Near+Action+Accessory+Group+Collage+Human+Occlusion+Info+Blur,data = df) # ~. 는 나머지 모든 변수에 대해 회귀분석 실시
summary(model3)


Call:
lm(formula = Pawpularity ~ Subject.Focus + Face + Near + Action + 
    Accessory + Group + Collage + Human + Occlusion + Info + 
    Blur, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-37.547 -13.057  -5.057   8.338  65.075 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)   37.21061    0.87130  42.707   <2e-16 ***
Subject.Focus -1.33904    1.27202  -1.053   0.2925    
Face           0.54534    0.71557   0.762   0.4460    
Near           0.30118    0.65787   0.458   0.6471    
Action        -0.23190    2.08245  -0.111   0.9113    
Accessory      1.15911    0.82999   1.397   0.1626    
Group          1.24299    0.65882   1.887   0.0592 .  
Collage        0.20131    1.12208   0.179   0.8576    
Human          0.33834    0.72922   0.464   0.6427    
Occlusion     -0.08618    0.71892  -0.120   0.9046    
Info          -0.72622    0.99434  -0.730   0.4652    
Blur          -1.86008    0.81194  -2.291   0.0220 *  
---
Signif. codes: 

Multiple R-squared 값이 너무 낮아서 설명력이 매우 부족함